<a href="https://colab.research.google.com/github/zzyy-gh/stonks-lambda-/blob/main/stonks_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# scrape stock data daily from finviz
# data: tkr, pb, sgqq, s, f, (sf), si, sti, fsh, de, opm, prfm, weekc, monc, quartc, halfc, yearc, b, weekv, monv, cfo, g, p, dayc, v
# run on UTC time 0100, 4 hours after NYC: use yesterday's date in utc to get latest market data
import requests
from datetime import datetime, timedelta
import time
from multiprocessing import Process, Manager
import json
import boto3
from bs4 import BeautifulSoup

client = boto3.client('dynamodb')

# ---------- functions ----------

def cleanData(rows, fsList):
    
    def cleanText(dataName, data):
        try:
            cleanItem['PutRequest']['Item'][dataName] = {'S': data}
            cleanFsItem[dataName] = data
        except:
            pass
        return

    def cleanSimpleFloat(dataName, data):
        try:
            if data != '-':
                cleanItem['PutRequest']['Item'][dataName] = {'N': data}
                cleanFsItem[dataName] = float(data)
        except:
            pass
        return
    
    def cleanCommaInt(dataName, data):
        try:
            if data != '-':
                cleanItem['PutRequest']['Item'][dataName] = {'N': data.replace(',', '')}
                cleanFsItem[dataName] = float(data.replace(',', ''))
        except:
            pass
        return
    
    def cleanPercFloat(dataName, data):
        try:
            if data != '-':
                cleanItem['PutRequest']['Item'][dataName] = {'N': data[:-1]}
                cleanFsItem[dataName] = float(data[:-1])
        except:
            pass
        return
    
    def cleanMBFloat(dataName, data):
        try:
            if data != '-':
                if data[-1] == 'M':
                    cleanItem['PutRequest']['Item'][dataName] = {'N': str(float(data[:-1]) * 1000000)}
                    cleanFsItem[dataName] = float(data[:-1]) * 1000000
                elif data[-1] == 'B':
                    cleanItem['PutRequest']['Item'][dataName] = {'N': str(float(data[:-1]) * 1000000000)} 
                    cleanFsItem[dataName] = float(data[:-1]) * 1000000000
        except:
            pass
        return
    
    date = (datetime.now() - timedelta(1)).strftime('%Y%m%d')
    pageCleanList = []
    rawList = []
    
    # extract raw data
    for row in rows:
        td = row.find_all('td')
        rawList.append([x.text for x in td])
        
    # save cleaned data
    for item in rawList:
        cleanItem = {'PutRequest':{'Item':{}}}
        cleanFsItem = {}
        cleanText('tkr', item[0])
        cleanSimpleFloat('pb', item[1])
        cleanPercFloat('sgqq', item[2])
        cleanMBFloat('s', item[3])
        cleanMBFloat('f', item[4])
        cleanPercFloat('si', item[5])
        cleanPercFloat('sti', item[6])
        cleanPercFloat('fsh', item[7])
        cleanSimpleFloat('de', item[8])
        cleanPercFloat('opm', item[9])
        cleanPercFloat('prfm', item[10])
        cleanPercFloat('weekc', item[11])
        cleanPercFloat('monc', item[12])
        cleanPercFloat('quartc', item[13])
        cleanPercFloat('halfc', item[14])
        cleanPercFloat('yearc', item[15])
        cleanSimpleFloat('b', item[16])
        cleanPercFloat('weekv', item[17])
        cleanPercFloat('monv', item[18])
        cleanPercFloat('cfo', item[19])
        cleanPercFloat('g', item[20])
        cleanSimpleFloat('p', item[21])
        cleanPercFloat('dayc', item[22])
        cleanCommaInt('v', item[23])

        cleanItem['PutRequest']['Item']['uuid'] = {'S': date + item[0]}
        cleanFsItem['uuid'] = date + item[0]
        cleanItem['PutRequest']['Item']['date'] = {'N': date}
        cleanFsItem['date'] = int(date)

        pageCleanList.append(cleanItem)
        fsList.append(cleanFsItem)
    
    return pageCleanList
    
def upload(cleanList):
    strCleanList = [ [] for _ in range((len(cleanList) - 1) // 25 + 1) ]
    unprocList = []
    for i, data in enumerate(cleanList):
        strCleanList[i // 25].append(data) 
    for batch in strCleanList:
        try: 
            response = client.batch_write_item(
                RequestItems= {
                    'ticker_meta': batch
                }
            )
            if bool(response['UnprocessedItems']):
                print(response)
                unprocList.extend(response['UnprocessedItems']['ticker_meta'])
        except Exception as e:
            print(batch[0])
            print(e)
    if len(unprocList) > 0:
        print(str(len(unprocList)) + ' unprocessed items.')
        upload(unprocList)
    return
    
def endChecker1(nicePage, stopPage):
    nextBtn = nicePage.find_all('a', class_='tab-link',string='next')
    stopper = nicePage.find_all('a', class_='tab-link', string=stopPage)
    
    if len(stopper) > 0 or len(nextBtn) == 0:
        return True
    return False

def endChecker2(nicePage, stopPage):
    nextBtn = nicePage.find_all('a', class_='tab-link',string='next')

    if len(nextBtn) == 0:
        return True
    return False
    
def scrape(startIndex, stopPage, finvizUrl1, finvizUrl2, faker, rowIncrement, finalList, fsList):

    # data and setup
    row = startIndex
    cleanList = []
    if stopPage == 'end':
        isLastPage = endChecker2
    else:
        isLastPage = endChecker1
    
                
    # iterate through all the webpages to obtain and clean data
    start = time.time()
    while True:

        # fetch webpage
        try:
            url = finvizUrl1 + str(row) + finvizUrl2
            page = requests.get(url, headers=faker)
        except:
            print('Failed to retrieve row ' + str(row) + '.')
            break

        # get page rows
        nicePage = BeautifulSoup(page.content, 'html.parser')
        rows = nicePage.find_all('tr', class_='table-dark-row-cp') + nicePage.find_all('tr', class_='table-light-row-cp')

        # if ads pops up, refresh
        if len(rows) == 0:
            # fetch webpage
            try:
                print('Retrieving refreshed page of row: ' + str(row) + '.')
                page = requests.get(url, headers=faker)
            except:
                print('Failed to retrieve refreshed page of row: ' + str(row) + '.')
                return pageCleanList
            nicePage = BeautifulSoup(page.content, 'html.parser')
            rows = nicePage.find_all('tr', class_='table-light-row-cp') + nicePage.find_all('tr', class_='table-dark-row-cp')
        
        if len(rows) == 0:
            print('Possibly screwed by ads. (page of row: ' + str(row) + ')')
        
        if len(rows) != 0:
            cleanList.extend(cleanData(rows, fsList))

        # exit loop if on the last page, else continue to next page
        if isLastPage(nicePage, stopPage):
            print('Data scraping has ended. The first row number of the last page is ' + str(row) + '.')
            break
        else:
            row += rowIncrement
            continue   
    print(str(time.time() - start) + ' seconds have elapsed for scraping. (last row ' + str(row) + ')')

    # return function if nothing is scraped
    if len(cleanList) == 0:
        print('No data is scraped. (for startIndex: ' + str(startIndex) + ')')
    
    finalList.extend(cleanList)
    return

def uploadFs(prepList, fsHttps):
    payload = json.dumps({
        'data': prepList
    })
    headers = {
      'Content-Type': 'application/json'
    }

    try:
        start = time.time()
        response = requests.request("POST", fsHttps, headers=headers, data=payload)
        print(str(response.content))
        print(str(time.time() - start) + ' seconds have elapsed for firestore upload.')
    except:
        pass
    return
    
# ---------- main function ----------

def lambda_handler(event, context):
    
    # ---------- global variables ----------

    marketIsOpen = False
    yesterday = datetime.now() - timedelta(1)
    date = yesterday.strftime('%Y-%m-%d')
    month = yesterday.strftime('%m')
    year = yesterday.strftime('%Y')
    calendarApi = 'https://sandbox.tradier.com/v1/markets/calendar'
    fsHttps = 'https://asia-southeast2-stonks-810ca.cloudfunctions.net/addSrapedMeta'
    calendarApiToken = 't0XrEyArrcq6EJZAAbP6zbZDl9FA'
    finvizUrl1 = 'https://finviz.com/screener.ashx?v=152&o=ticker&r='
    finvizUrl2 = '&c=1,11,23,24,25,26,28,30,38,40,41,42,43,44,45,46,48,50,51,60,61,65,66,67'
    faker = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    rowIncrement = 20
    startIndex1 = 1
    startIndex2 = 4101
    stopPage1 = '205'
    stopPage2 = 'end'

    # ---------- check if market is open, else return ----------

    calendar = requests.get(calendarApi, 
                     headers={'Authorization': calendarApiToken, 'Accept': 'application/json'},
                     params={'month': month, 'year': year})

    dayListInThisMonth = calendar.json()['calendar']['days']['day']
    try:
        for day in dayListInThisMonth:
            if day['date'] == date:
                if day['status'] == 'open':
                    marketIsOpen = True
                    break
        if marketIsOpen:
            print('Market is open.')
        else:
            print('Market is closed.')        
    except:
        print('Failed to check market\'s status.')

    if not marketIsOpen:
        return

    # ---------- scrape and upload data from finviz ----------
    
    with Manager() as manager:
        # creating processes
        start = time.time()
        finalList = manager.list()
        fsList = manager.list()
        p1 = Process(target=scrape, args=(startIndex1, stopPage1, finvizUrl1, finvizUrl2, faker, rowIncrement, finalList, fsList,))
        p2 = Process(target=scrape, args=(startIndex2, stopPage2, finvizUrl1, finvizUrl2, faker, rowIncrement, finalList, fsList,))

        # starting process 1
        p1.start()
        # starting process 2
        p2.start()

        # wait until process 1 is finished
        p1.join()
        # wait until process 2 is finished
        p2.join()
        print(str(time.time() - start) + ' seconds have elapsed for scraping in total.')

        # upload to dynamodb and firestore
        print(str(len(finalList)) + ' rows to be uploaded.')
        start = time.time()
        p1 = Process(target=upload, args=(finalList, ))
        prepList = fsList._getvalue()
        p2 = Process(target=uploadFs, args=(prepList, fsHttps, ))
        
        # starting process 1
        p1.start()
        # starting process 2
        p2.start()

        # wait until process 1 is finished
        p1.join()
        # wait until process 2 is finished
        p2.join()
        print(str(time.time() - start) + ' seconds have elapsed for both uploads.')

    return 


